# Question 1:
How many employees spent above their given stipend? Extract the name of the employee, the assigned stipend, and the actual amount spent.

In [1]:
import psycopg2
import pandas as pd

In [3]:
conn = psycopg2.connect(
    host='localhost'
    , user = 'admin'
    , password = 'Pa$$w0rd'
    , database = 'bissan'
)
cursor = conn.cursor()

In [4]:
query = """
    SELECT DISTINCT name, entitled_bonus AS stipend
    FROM employees e
    LEFT JOIN departments d
        ON e.department_id = d._id
"""

cursor.execute(query)
all_employees_stipend = cursor.fetchall()

In [5]:
employees_df = pd.DataFrame(all_employees_stipend, columns=['name', 'stipend'])

In [6]:
employees_df[employees_df['name'].duplicated()]

,name,stipend
14,John Bentley,6086
133,Thomas Jordan,6086
146,Sean Renauer,6832
155,Mitsuko Herrera,6913
164,Catherine Brewer,6086
175,Sean Renauer,6913
320,Russ Hamill,6913
333,Dimitry Ruvin,6913
343,Darren Crandell,6832
375,Mark Sheelor,7377


In [7]:
travels_dataset = pd.read_csv('cleaned_travels_dataset.csv') # read the csv
travels_dataset = travels_dataset.drop('Unnamed: 0', axis=1) # remove the column "Unnamed: 0"

In [16]:
travels_dataset

,Employee,Destination,Travel Start Date,Travel End Date,Actual Total Expenses,Purpose Of Travel,State,City,Firstname,Lasttname,Lastname
0,Aaron Salter,"OCEAN CITY, MD",2017-05-31,2017-06-02,644.10,Conference,MD,OCEAN CITY,Aaron,Salter,Salter
1,Abigail Ratnofsky,"SAN DIEGO, CA",2016-01-24,2016-01-29,2962.70,Conference,CA,SAN DIEGO,Abigail,Ratnofsky,Ratnofsky
2,Adam Kisthardt,"ORLANDO, FL",2018-10-04,2018-10-09,1891.19,Conference,FL,ORLANDO,Adam,Kisthardt,Kisthardt
3,Adam P. Jones,"OCEAN CITY, MD",2016-06-19,2016-06-21,324.36,Conference,MD,OCEAN CITY,Adam,P.,Jones
4,Alan Butsch,"SAN DIEGO, CA",2019-11-19,2019-11-22,2141.51,Conference,CA,SAN DIEGO,Alan,Butsch,Butsch
...,...,...,...,...,...,...,...,...,...,...,...
413,Warren Jensen,"OCEAN CITY, MD",2017-11-06,2017-11-09,502.45,Conference,MD,OCEAN CITY,Warren,Jensen!,Jensen!
414,Warp Jensen,"OCEAN CITY, MD",2019-03-25,2019-03-26,78.00,Conference,MD,OCEAN CITY,Warp,Jensen,Jensen
415,Whitney Kujawa,"OCEAN CITY, MD",2017-11-06,2017-11-09,567.00,Conference,MD,OCEAN CITY,Whitney,Kujawa,Kujawa
416,William Kinna,"CHARLOTTE, NC",2017-05-21,2017-05-26,2163.67,Conference,NC,CHARLOTTE,William,Kinna,Kinna


In [9]:
employees_expense = travels_dataset[['Employee' ,'Actual Total Expenses']] # retrieve Employee and Actual Total Expenses

In [10]:
employees_expense[employees_expense['Employee'].duplicated()]

,Employee,Actual Total Expenses
26,Barbara Natoli,991.12
34,Bill Ferretti,353.22
52,Catherine Brewer,4925.24
80,Darren Crandell,1995.22
99,Dimitry Ruvin,2195.32
189,John Bentley,1119.22
229,Kevin Parker,1899.98
255,Lissa Bales,258.64
259,Mackell Heflin,2964.20
274,Mark Sheelor,1411.34


In [11]:
employees_expense = employees_expense.groupby(by='Employee').sum() # get the total expense for each employee

In [ ]:
employees_df = employees_df.drop_duplicates(subset='name') # drop duplicated names

In [12]:
employees_expense = employees_expense.reset_index() # reset index to pandas' default

In [13]:
employees_df

,name,stipend
0,Robert Green,6832
1,James Sawyer,6832
2,Dieter Klinger,7377
3,Chase Fabrizio,6913
4,Scott Davis,6913
...,...,...
408,Lisa Killen,6086
409,Jason Giza,6832
410,Dana Stroman,7377
411,Joseph Webster,6832


In [14]:
employees_expense

,Employee,Actual Total Expenses
0,Aaron Salter,644.10
1,Abigail Ratnofsky,2962.70
2,Adam Kisthardt,1891.19
3,Adam P. Jones,324.36
4,Alan Butsch,2141.51
...,...,...
396,Warren Jensen,502.45
397,Whitney Kujawa,567.00
398,William Kinna,2163.67
399,Wins Hinde,352.00


In [15]:
# join the two DFs to obtain result

employees_expense.merge(employees_df, how='inner', left_on='Employee', right_on='name')

,Employee,Actual Total Expenses,name,stipend
0,Aaron Salter,644.10,Aaron Salter,7377
1,Abigail Ratnofsky,2962.70,Abigail Ratnofsky,6086
2,Adam Kisthardt,1891.19,Adam Kisthardt,7377
3,Adam P. Jones,324.36,Adam P. Jones,6832
4,Alan Butsch,2141.51,Alan Butsch,6913
...,...,...,...,...
408,Warren Jensen,502.45,Warren Jensen,6086
409,Whitney Kujawa,567.00,Whitney Kujawa,6832
410,William Kinna,2163.67,William Kinna,7377
411,Wins Hinde,352.00,Wins Hinde,6086
